In [11]:
from bs4 import BeautifulSoup
import requests
import networkx as nx
from urllib.parse import urljoin
from tqdm import tqdm

## Pull the data that shows the relationships between types

In [2]:
# Target web page
home_url = "https://pokemondb.net"

#Connection to web page
response = requests.get(home_url)
print(response.status_code)

# Convert the response HTLM string into a python string
html = response.text

soup = BeautifulSoup(response.content, 'lxml')

200


In [3]:
type_links = soup.find_all("a", class_="type-icon", href=True)

In [4]:
type_info = {}
for type_link in type_links:
    pk_type = type_link.text
    url = urljoin(home_url, type_link["href"])
    type_info[pk_type] = {"url": url}

In [5]:
type_info["Normal"]

{'url': 'https://pokemondb.net/type/normal'}

In [8]:
for pk_type, info in tqdm(type_info.items(), total=len(type_info)):
    response = requests.get(info["url"])

    # Convert the response HTLM string into a python string
    html = response.text
    soup = BeautifulSoup(response.content, 'lxml')
    
    # ok going to be gross here, basically want to just loop through each pair of these since they go together like that
    # for example, for [a, b, c, d], want [a, b], [c, d]
    all_ps = soup.find_all("div", class_="grid-col")[1].find_all('p')
    iterator = iter(all_ps)
    num_rows = len(all_ps)

    structs = []
    try:
        while True:
            list_type_text = next(iterator).text
            if "are super-effective" in list_type_text:
                list_type = "super_effective"
            elif "are not very effective" in list_type_text:
                list_type = "not_very_effective"
            elif "have no effect" in list_type_text:
                list_type = "no_effect"
            else:
                list_type = "unknown"

            pk_types = next(iterator).text.strip().split(" ")
            structs.append((list_type, pk_types))
    except StopIteration:
        pass
    
    type_info[pk_type]["attacks"] = structs

100%|█████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:05<00:00,  3.42it/s]


In [9]:
type_info["Rock"]

{'url': 'https://pokemondb.net/type/rock',
 'attacks': [('super_effective', ['Fire', 'Ice', 'Flying', 'Bug']),
  ('not_very_effective', ['Fighting', 'Ground', 'Steel'])]}

## Now create the graph

In [12]:
graph = nx.DiGraph()

In [14]:
for pk_type, info in type_info.items():
    for effectiveness, pk_types in info["attacks"]:
        if effectiveness == "super_effective":
            for effective_pk_type in pk_types:
                graph.add_edge(pk_type, effective_pk_type)

In [17]:
graph.edges

OutEdgeView([('Fire', 'Grass'), ('Fire', 'Ice'), ('Fire', 'Bug'), ('Fire', 'Steel'), ('Grass', 'Water'), ('Grass', 'Ground'), ('Grass', 'Rock'), ('Ice', 'Grass'), ('Ice', 'Ground'), ('Ice', 'Flying'), ('Ice', 'Dragon'), ('Bug', 'Grass'), ('Bug', 'Psychic'), ('Bug', 'Dark'), ('Steel', 'Ice'), ('Steel', 'Rock'), ('Steel', 'Fairy'), ('Water', 'Fire'), ('Water', 'Ground'), ('Water', 'Rock'), ('Ground', 'Fire'), ('Ground', 'Electric'), ('Ground', 'Poison'), ('Ground', 'Rock'), ('Ground', 'Steel'), ('Rock', 'Fire'), ('Rock', 'Ice'), ('Rock', 'Flying'), ('Rock', 'Bug'), ('Electric', 'Water'), ('Electric', 'Flying'), ('Flying', 'Grass'), ('Flying', 'Fighting'), ('Flying', 'Bug'), ('Dragon', 'Dragon'), ('Fighting', 'Normal'), ('Fighting', 'Ice'), ('Fighting', 'Rock'), ('Fighting', 'Dark'), ('Fighting', 'Steel'), ('Dark', 'Psychic'), ('Dark', 'Ghost'), ('Poison', 'Grass'), ('Poison', 'Fairy'), ('Fairy', 'Fighting'), ('Fairy', 'Dragon'), ('Fairy', 'Dark'), ('Psychic', 'Fighting'), ('Psychic', 'Po

## Now we need to find 6 pokemon who are super effective against 3 types to be able to have a super effective pokemon against any pokemon type

We calculate this by choosing six pokemon types out of a bag, and check if they have all eighteen types covered

In [33]:
import itertools

combos = {}
for pk_type_combo in itertools.combinations(graph.nodes, 6):
    effective_set = set()
    for pk_type in pk_type_combo:
        for effective_pk_type in graph[pk_type].keys():
            effective_set.add(effective_pk_type)
            
    combos[pk_type_combo] = len(effective_set)
    
print(f"Number of combinations: {len(combos)}")
print(f"Num combos that have all 18 types: {len([x for k,v in combos.items() if v == 18])}")

Number of combinations: 18564
Num combos that have all 18 types: 0


In [38]:
from collections import defaultdict

hist_values = defaultdict(int)

for k, v in combos.items():
    hist_values[v] += 1
    
hist_values

defaultdict(int,
            {13: 4176,
             15: 936,
             12: 4513,
             14: 2705,
             11: 3261,
             10: 1827,
             16: 183,
             9: 719,
             8: 189,
             7: 42,
             17: 10,
             6: 3})

### Looks like there are only 10 combos that cover only max 17 types. What are those?

In [39]:
for k,v in combos.items():
    if v == 17:
        print(k)

('Grass', 'Ice', 'Ground', 'Flying', 'Fighting', 'Dark')
('Grass', 'Ice', 'Ground', 'Flying', 'Fighting', 'Ghost')
('Grass', 'Ground', 'Rock', 'Dark', 'Poison', 'Fairy')
('Grass', 'Ground', 'Rock', 'Poison', 'Fairy', 'Ghost')
('Ice', 'Steel', 'Ground', 'Flying', 'Fighting', 'Dark')
('Ice', 'Steel', 'Ground', 'Flying', 'Fighting', 'Ghost')
('Ice', 'Ground', 'Electric', 'Flying', 'Fighting', 'Dark')
('Ice', 'Ground', 'Electric', 'Flying', 'Fighting', 'Ghost')
('Ice', 'Ground', 'Flying', 'Fighting', 'Dark', 'Poison')
('Ice', 'Ground', 'Flying', 'Fighting', 'Poison', 'Ghost')


## Wait a second, I believe my thinking is flawed here. We can always ADD more moves to cover the 18. What we really might want to do is choose the six that minimize exposure. Aka, choose six pokemon that will have the LEAST number of types that are super effective against them

In [40]:
reversed_graph = nx.DiGraph()

for edge in graph.edges:
    reversed_graph.add_edge(edge[1], edge[0])

In [41]:
import itertools

reversed_combos = {}
for pk_type_combo in itertools.combinations(graph.nodes, 6):
    effective_set = set()
    for pk_type in pk_type_combo:
        for effective_pk_type in reversed_graph[pk_type].keys():
            effective_set.add(effective_pk_type)
            
    reversed_combos[pk_type_combo] = len(effective_set)
    
print(f"Number of combinations: {len(reversed_combos)}")

Number of combinations: 18564


In [42]:
from collections import defaultdict

reversed_hist_values = defaultdict(int)

for k, v in reversed_combos.items():
    reversed_hist_values[v] += 1
    
reversed_hist_values

defaultdict(int,
            {12: 5052,
             11: 4146,
             10: 2161,
             14: 1813,
             13: 3794,
             15: 539,
             16: 100,
             9: 779,
             8: 161,
             7: 15,
             17: 4})

### Looks like there are some where there exposure is limited to only 7 types! What are they?

In [43]:
for k,v in reversed_combos.items():
    if v == 7:
        print(k)

('Fire', 'Ice', 'Bug', 'Steel', 'Electric', 'Normal')
('Fire', 'Ice', 'Steel', 'Rock', 'Electric', 'Normal')
('Fire', 'Ice', 'Steel', 'Electric', 'Normal', 'Poison')
('Fire', 'Ice', 'Steel', 'Electric', 'Normal', 'Fairy')
('Fire', 'Bug', 'Steel', 'Electric', 'Normal', 'Poison')
('Fire', 'Water', 'Ground', 'Electric', 'Flying', 'Normal')
('Fire', 'Water', 'Ground', 'Electric', 'Flying', 'Poison')
('Ice', 'Bug', 'Steel', 'Electric', 'Normal', 'Poison')
('Ice', 'Bug', 'Steel', 'Electric', 'Normal', 'Fairy')
('Ice', 'Steel', 'Electric', 'Normal', 'Poison', 'Fairy')
('Bug', 'Steel', 'Electric', 'Fighting', 'Normal', 'Poison')
('Steel', 'Electric', 'Fighting', 'Normal', 'Dark', 'Poison')
('Steel', 'Electric', 'Normal', 'Dark', 'Psychic', 'Ghost')
('Steel', 'Electric', 'Normal', 'Poison', 'Psychic', 'Ghost')
('Electric', 'Normal', 'Dark', 'Poison', 'Psychic', 'Ghost')


### What what a combination! This is quite interesting indeed...Now, can we see how much overlap these types have? Take the first group for example. Say the other team has a pokemon that is effective against Fire. How many other types in our group will also be exposed?